In [ ]:
# Imports

import os
import shutil
import sys
from pathlib import Path
from typing import List

import h5py
import matplotlib.pyplot as plt
import numpy as np
import pyFAI
from pyFAI.calibrant import CALIBRANT_FACTORY
from pyFAI.goniometer import SingleGeometry
from pyFAI.gui import jupyter
from tqdm.notebook import tqdm

plt.rcParams["figure.figsize"] = 13, 7

In [ ]:
# Papermill things
inpath: str = None  # Incoming nexus file _auto_
outpath: str = None  # The outgoing nexus file _auto_
dataset_path: str = "entry/data"  # The path to the data in the nexus file _auto_
# Where in the nexus file the poni path is _required_
poni_tree_location: str = "/entry1/poni"
n_pts: str = None  # Initial Poni File,full path to the poni file _required_
host: str = None  # the host on which the GDA server is running _auto_

In [ ]:
inpath = Path("/inputs").joinpath(inpath)
outpath = Path("/outputs").joinpath(outpath)
xy_tth_filepath = outpath.with_stem(f"{outpath.stem}_tth").with_suffix(".xy")
xy_q_filepath = outpath.with_stem(f"{outpath.stem}_q").with_suffix(".xy")

In [ ]:
# Daq Messenger
sys.path.append("/dls_sw/apps/daq-messenger")
daq = None
try:
    from daqmessenger import DaqMessenger
    daq = DaqMessenger(host)
    daq.connect()
except Exception as e:
    print("no messenger")

# Integration

In [ ]:
try:
    with h5py.File(inpath, 'r') as f:
        # poni things
        poni_filepath = f.get(poni_tree_location)[()].decode("utf-8")
        print("the poni filepath is {}".format(poni_filepath))
        ai = pyFAI.load(poni_filepath)

        # get the size / shape of the scan
        data_all = f.get(dataset_path)
        scan_dimensions = data_all.shape[:-2]

        # set up the result arrays
        results_q = results_tth = np.zeros(list(scan_dimensions) + [n_pts])

        # iterate
        for index in tqdm(np.ndindex(scan_dimensions), total=np.product(scan_dimensions)):
            data = data_all[index][()].squeeze()

            result_tth = ai.integrate1d(
                data, n_pts, unit=pyFAI.units.TTH_DEG, radial_range=[0, 48])
            results_tth[index] = result_tth.intensity
            xy_tth = xy_tth_filepath.with_stem(
                xy_tth_filepath.stem + "_" + "_".join([str(x).zfill(4) for x in index]))
            np.savetxt(xy_tth, np.transpose(result_tth))

            result_q = ai.integrate1d(
                data, n_pts, unit=pyFAI.units.Q_A, radial_range=[0, 10.4])
            results_q[index] = result_q.intensity
            xy_q = xy_q_filepath.with_stem(
                xy_q_filepath.stem + "_" + "_".join([str(x).zfill(4) for x in index]))
            np.savetxt(xy_q, np.transpose(result_q))

except Exception as e:
    print("could not extract initial information from {}".format(inpath))
    raise e

In [ ]:
ax = jupyter.plot1d(result_tth)

In [ ]:
# we could also usefully dump this stuff in a nexus file
with h5py.File(str(outpath), 'w') as f:
    # -- entry
    nxentry = f.create_group("entry")
    nxentry.attrs["NX_class"] = "NXentry"

    # -- data
    nxdata = nxentry.create_group("data")
    nxdata.attrs["NX_class"] = "NXdata"
    nxdata.attrs["signal"] = "counts"
    nxdata.attrs["axes"] = "two_theta"
    nxdata.attrs["two_theta_indices"] = [
        len(scan_dimensions),
    ]

    tth = nxdata.create_dataset("two_theta", data=result_tth.radial)
    tth.attrs["units"] = "degrees"
    counts = nxdata.create_dataset("counts", data=results_tth)
    counts.attrs["units"] = "counts"

    q = nxdata.create_dataset("q", data=result_q.radial)
    q.attrs["units"] = "inverse_angstroms"
    counts_q = nxdata.create_dataset("counts_q", data=results_q)
    counts_q.attrs["units"] = "counts"

    nxentry['raw'] = h5py.ExternalLink(inpath, "/")

In [ ]:
# try to send the pattern back to the client
if daq:
    daq.send_file(str(outpath))
    magic_path = str(inpath.parent / Path(".ispyb") /
                     inpath.stem) + "_pixium_hdf/data.dat"
    shutil.copy2(xy_tth, magic_path)